In [ ]:
import math
import random
import numpy as np
import networkx as nx
from pathlib import Path


data_folder = Path.cwd() / '../../Data'

### Generate unique graphs with 3-7 nodes.

- 2 graphs for n = 3
- 6 graphs for n = 4
- 21 graphs for n = 5
- 112 graphs for n = 6
- 853 graphs for n = 7

TOTAL: 994

In [ ]:
import codetiming
import unique_graphs

total = 0
gs = None

for i in range(3, 8 + 1):
    all_graphs = {}
    with codetiming.Timer():
        gs = unique_graphs.make_graphs_faster(i, gs)
    print(f"Graphs built: {len(gs)}\n")
    total += len(gs)

    for k, edges in enumerate(gs):
        g = nx.from_edgelist(edges)
        all_graphs[f'G{i},{k}'] = nx.to_numpy_array(g)

    with open(data_folder / f'UniqueGraphs_{i}.npz', 'wb') as stream:
        np.savez(stream, **all_graphs)

print(f"Files generated. Total {total} graphs.")

In [ ]:
from unique_graphs import make_graphs_filtered, filter, make_graphs
import codetiming
import itertools

# Normal approach
with codetiming.Timer():
    gs5_old = make_graphs_filtered(5)

with codetiming.Timer():
    gs6_old = make_graphs_filtered(6)
    
gs6_old_raw = make_graphs(6)
    
    
# New approach
n = 6
with codetiming.Timer():
    gs6_new_raw = []
    for g in gs5_old:
        for i in range(n-1):
            for comb in itertools.combinations(range(n-1), i+1):    
                temp = []
                for p in comb:
                    temp.append((p, n-1))
                gs6_new_raw.append(sorted(g + temp))

    gs6_new = filter(gs6_new_raw, 6)

### Generate radnom graphs

Using nx.connected_watts_strogatz_graph
- 50 graphs with n = 4, 5, ..., 29 (1300 in total)
- 50 graphs with n = 30, 40, ..., 100 (400 in total)

TOTAL: 1700

In [ ]:
all_graphs = {}

for n in range(5, 31):
    print(f"Generating 50 graphs with {n} nodes.")
    for i in range(50):
        G = nx.connected_watts_strogatz_graph(n, random.randint(2, int(math.sqrt(n))), random.random())
        all_graphs[f'R{n},{i}'] = nx.to_numpy_array(G)

for n in range(30, 101, 10):
    print(f"Generating 50 graphs with {n} nodes.")
    for i in range(50):
        G = nx.connected_watts_strogatz_graph(n, random.randint(2, int(math.sqrt(n))), random.random())
        all_graphs[f'R{n},{i}'] = nx.to_numpy_array(G)

with open(data_folder / f'RandomGraphs.npz', 'wb') as stream:
        np.savez(stream, **all_graphs)